In [54]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.utils import check_array
import numpy as np
from numba import jit, vectorize, float64, int64
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.externals import joblib
from math import sqrt
from IPython import display
import dash
import dash_core_components as dcc
import dash_html_components as html
import warnings
import plotly.graph_objs as go
from plotly import tools
from plotly.figure_factory import create_2d_density
from plotly.graph_objs import graph_objs

In [55]:
warnings.filterwarnings('ignore')

In [56]:
rf = joblib.load('./data/rfc_trained.pkl')
train = pd.read_feather('./data/xc_train.feather')
valid = pd.read_feather('./data/xc_valid.feather')
y_train = pd.read_csv('./data/yc_train.csv')
y_valid = pd.read_csv('./data/yc_valid.csv')

# show the Dash plot

In [57]:
def show_app(app,  # type: dash.Dash
             port=9999,
             width=700,
             height=350,
             offline=True,
             style=True,
             **dash_flask_kwargs):
    """
    Run the application inside a Jupyter notebook and show an iframe with it
    :param app:
    :param port:
    :param width:
    :param height:
    :param offline:
    :return:
    """
    url = 'http://localhost:%d' % port
    iframe = '<iframe src="{url}" width={width} height={height}></iframe>'.format(url=url,
                                                                                  width=width,
                                                                                  height=height)
    display.display_html(iframe, raw=True)
    if offline:
        app.css.config.serve_locally = True
        app.scripts.config.serve_locally = True
    if style:
        external_css = ["https://fonts.googleapis.com/css?family=Raleway:400,300,600",
                        "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                        "http://getbootstrap.com/dist/css/bootstrap.min.css", ]

        for css in external_css:
            app.css.append_css({"external_url": css})

        external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
                       "https://cdn.rawgit.com/plotly/dash-app-stylesheets/a3401de132a6d0b652ba11548736b1d1e80aa10d/dash-goldman-sachs-report-js.js",
                       "http://getbootstrap.com/dist/js/bootstrap.min.js"]

        for js in external_js:
            app.scripts.append_script({"external_url": js})

    return app.run_server(debug=False,  # needs to be false in Jupyter
                          port=port)
                          #**dash_flask_kwargs)

In [58]:
app = dash.Dash()

In [59]:
rf = joblib.load('./data/rfc_trained.pkl')
train = pd.read_feather('./data/xc_train.feather')
valid = pd.read_feather('./data/xc_valid.feather')
y_train = pd.read_csv('./data/yc_train.csv',header=None).iloc[:,0]
y_valid = pd.read_csv('./data/yc_valid.csv',header=None).iloc[:,0]

def pred_to_hard(val, arr):
    arr[arr > val] = 1
    arr[arr <= val] = 0
    return arr


def hard_results(preds, target, value):
    hard = pred_to_hard(value, preds.copy())
    tp_ = np.sum(np.logical_and((hard == 1), (y_valid.values == 1)))
    tn_ = np.sum(np.logical_and((hard == 0), (y_valid.values == 0)))
    fp_ = np.sum(np.logical_and((hard == 1), (y_valid.values == 0)))
    fn_ = np.sum(np.logical_and((hard == 0), (y_valid.values == 1)))
    return (tp_,tn_,fp_,fn_)


def hard_pred_df(data, target, model):
    df = pd.DataFrame(columns=['Break','TP','TN', 'FP', 'FN'])
    preds = model.predict_proba(data)[:,1]
    for i,v in enumerate(np.arange(0,1.001,.001)):
        hard = pred_to_hard(v, preds.copy())
        tp_ = np.sum(np.logical_and((hard == 1), (y_valid.values == 1)))
        tn_ = np.sum(np.logical_and((hard == 0), (y_valid.values == 0)))
        fp_ = np.sum(np.logical_and((hard == 1), (y_valid.values == 0)))
        fn_ = np.sum(np.logical_and((hard == 0), (y_valid.values == 1)))
        df.loc[i] = {
                     'Break':v,
                     'TP':tp_,
                     'TN':tn_,
                     'FP':fp_,
                     'FN':fn_
                    }
    return df

In [60]:
hard_pred = hard_pred_df(valid,y_valid,rf)

hard_pred.head()

,Break,TP,TN,FP,FN
0,0.000,32195.0,40376.0,9969.0,0.0
1,0.001,32195.0,40376.0,9969.0,0.0
2,0.002,32195.0,40471.0,9874.0,0.0
3,0.003,32195.0,40547.0,9798.0,0.0
4,0.004,32195.0,40614.0,9731.0,0.0


In [82]:
import dash
import dash_html_components as html
import dash_core_components as dcc

preds = rf.predict_proba(valid)[:,1]

app = dash.Dash()
app.layout = html.Div([
    dcc.Slider(
        id='my-slider',
        min=0,
        max=1,
        step=0.01,
        value=.5
    ),
    html.Div(id='slider-output-container'),
    dcc.Graph(
    id='my-graph',
    )
])


@app.callback(
    dash.dependencies.Output('my-graph', 'figure'),
    [dash.dependencies.Input('my-slider', 'value')])
def update_output(value):
    results = hard_results(preds,y_valid,value)
    #return 'TP: %s, TN: %s, FP: %s, FN: %s' % (results[0],results[1],results[2],results[3])
    return {
            'data':[go.Scatter(x = [0,1],
                                 y =[results[0]+results[1], results[2]+results[3]])
                   ],
            'layout': {height :225, margin:{'l':} 
                      }
           }

In [ ]:
show_app(app,  # type: dash.Dash
             port=9999,
             width=1000,
             height=1000,
             offline=True,
             style=True)

 * Running on http://127.0.0.1:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-component-suites/dash_core_components/react-select@1.0.0-rc.3.min.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-component-suites/dash_core_components/react-virtualized@9.9.0.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-component-suites/dash_core_components/rc-slider@6.1.2.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-component-suites/dash_core_components/react-virtualized-select@3.1.0.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-component-suites/dash_core_components/react-dates@12.3.0.css?v=0.22.1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.12.1 HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2018 19:29:37] "GET /_dash-componen

(32095, 42577, 7768, 100)


[2018-04-30 19:29:46,399] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py",

(32195, 42555, 7790, 0)


[2018-04-30 19:29:55,167] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/imarobit-mac/anaconda2/envs/fastai/lib/python3.6/site-packages/flask/app.py",

(31611, 42705, 7640, 584)
